In [1]:
import pandas as pd
import numpy as np
import os
os.chdir(os.path.dirname(os.getcwd()))
print(os.getcwd())

%load_ext autoreload
%autoreload 2
%reload_ext autoreload


from resources.constants import *

pictures_df = pd.read_csv(PICTURE_TRIPLETS_CSV_PATH, sep=CSV_SEPARATOR)
outfits_df = pd.read_csv(OUTFITS_CSV_PATH, sep=CSV_SEPARATOR)
user_triplets_df = pd.read_csv(USER_ACTIVITY_TRIPLETS_CSV_PATH, sep=CSV_SEPARATOR)

c:\Users\kaborg15\Python_projects\Vibrent_Dataset_Collection


In [2]:
outfits_df["tag_categories"] = outfits_df["tag_categories"].apply(eval)
outfits_df["outfit_tags"] = outfits_df["outfit_tags"].apply(eval)

In [3]:
id_group_dict = outfits_df[["id", "group"]].to_dict(orient="records")
id_group_dict = {x["id"]: x["group"] for x in id_group_dict}

user_triplets_df["group"] = user_triplets_df["outfit.id"].map(id_group_dict)
user_triplets_df

,customer.id,outfit.id,meta.validFrom,derived.bookingTime,group
0,969,outfit.5c081909537b42239e465d2d615c705f,2023-04-27 12:46:35.184,2023-03-19 14:24:09.539,group.79d7302719c9325ecb7a4a140fb25ca1
1,209,outfit.c34969dd8b334064aa90bfb60c8ec308,2023-05-05 09:44:38.200,2023-01-17 13:36:53.371,group.8625d8e9c5b8da9b4793dd636ac9afc9
2,209,outfit.aef4cc93eebf40ca8820790deb7a8323,2023-05-05 09:44:38.200,2023-01-17 13:36:53.371,group.f006b0dd53fdf64104db43f9c8dfe872
3,2235,outfit.3de5df48a14b4a9aba6d8e41d11e9351,2022-11-22 03:04:56.144,2021-10-04 10:00:17.854,group.3222ea830704958953d7f63097e88143
4,2235,outfit.0eaa358af14e469894062591bd42f38b,2022-11-22 03:04:56.144,2021-10-04 10:00:17.854,group.702a9964f42e5d231868610b9ed1422a
...,...,...,...,...,...
64165,1426,outfit.5bedabce07614c608b03fbb369d9abe7,2022-11-22 13:11:49.877,2021-07-10 08:09:49.250,group.34598adaf25b5b74f05e182257329ec0
64166,358,outfit.bb35c791b2bfa191,2021-05-28 11:41:48.526,2021-04-19 06:52:08.332,group.8cd6c26aaa386a01ba422c0f47bf9a1e
64167,358,outfit.ad6f853b5c31b6f5,2021-05-28 11:41:48.526,2021-04-19 06:52:08.332,group.793eca0e71eafbf27c6f0bddad69e593
64168,1629,outfit.73393583c562454cbf1c00ce3cac0862,2023-11-25 07:48:26.991,2023-11-21 13:27:25.349,group.f75720d7b914feb2e6fabead7c3b3525


In [4]:
user_orders_df = user_triplets_df.groupby("customer.id").agg({"outfit.id": list, "group":list, "meta.validFrom":list, "derived.bookingTime":list}).reset_index()
user_orders_df["num_orders"] = user_orders_df["outfit.id"].apply(lambda x: len(x))
user_orders_df = user_orders_df[user_orders_df["num_orders"] > 1]
user_orders_df["num_orders"].describe()

count    2216.000000
mean       28.942690
std        41.905773
min         2.000000
25%         5.000000
50%        13.000000
75%        35.000000
max       356.000000
Name: num_orders, dtype: float64

In [5]:
import numpy as np

def leave_one_out_split(outfit_ids, groups, derived_booking_times):
    outfit_ids = np.array(outfit_ids)
    groups = np.array(groups)
    derived_booking_times = np.array(derived_booking_times)
    sorted_indices = np.argsort(derived_booking_times)
    return outfit_ids[sorted_indices[:-1]], outfit_ids[sorted_indices[-1]], groups[sorted_indices[:-1]], groups[sorted_indices[-1]], derived_booking_times[sorted_indices[:-1]], derived_booking_times[sorted_indices[-1]]

def leave_one_out_split_unique(outfit_ids, groups, derived_booking_times):
    outfit_ids = np.array(outfit_ids)
    groups = np.array(groups)
    derived_booking_times = np.array(derived_booking_times)
    
    sorted_indices = np.argsort(derived_booking_times)
    sorted_outfit_ids = outfit_ids[sorted_indices]
    sorted_groups = groups[sorted_indices]
    sorted_booking_times = derived_booking_times[sorted_indices]
    
    unique_groups, counts = np.unique(sorted_groups, return_counts=True)
    
    single_count_indices = np.where(counts == 1)[0]
    if len(single_count_indices) == 0:
        print(f"No unique outfit found with groups {groups}")
        return None
    
    unique_group = unique_groups[single_count_indices[0]]
    unique_group_index = np.where(sorted_groups == unique_group)[0][0]
    remaining_indices = np.arange(len(sorted_groups)) != unique_group_index
    
    return (
        sorted_outfit_ids[remaining_indices], sorted_outfit_ids[unique_group_index],
        sorted_groups[remaining_indices], sorted_groups[unique_group_index],
        sorted_booking_times[remaining_indices], sorted_booking_times[unique_group_index]
    )


user_splits = user_orders_df.apply(lambda x: leave_one_out_split(x["outfit.id"], x["group"], x["derived.bookingTime"]), axis=1)
user_splits_df = pd.DataFrame(user_splits.tolist(), columns=["train_outfit_ids", "test_outfit_id", "train_group", "test_group", "train_booking_times", "test_booking_time"])
user_splits_unique = user_orders_df.apply(lambda x: leave_one_out_split_unique(x["outfit.id"], x["group"], x["derived.bookingTime"]), axis=1)
user_splits_unique_df = pd.DataFrame(user_splits_unique.tolist(), columns=["train_outfit_ids", "test_outfit_id", "train_group", "test_group", "train_booking_times", "test_booking_time"])
user_splits_unique_df = user_splits_unique_df.dropna()

No unique outfit found with groups ['group.8abe6af9eccc8b578c2ef59628f8b454'
 'group.96f4cce22d4a236e0652c67fc9b18d12'
 'group.8abe6af9eccc8b578c2ef59628f8b454'
 'group.96f4cce22d4a236e0652c67fc9b18d12']


In [6]:
user_splits_unique_df

,train_outfit_ids,test_outfit_id,train_group,test_group,train_booking_times,test_booking_time
0,"[outfit.a0358734a2b24bf6b531d86101378b7d, outf...",outfit.31d5bc7c60334b9889e9ed768f05e578,"[group.e7926cffbe8336c0f24ed413753ed407, group...",group.40372db1f00c37b946653061d4242bed,"[2020-11-22 21:27:04.340, 2020-11-22 21:27:04....",2020-12-03 16:33:14.331
1,"[outfit.cc2a4ea6b82044d2804ee26e593fbc00, outf...",outfit.2405f2beaf9241f590e020d163f6fb66,"[group.89c749588d0ce926c1a32fdabf649397, group...",group.09ecaaef60ad3c09be4a28671378255c,"[2024-01-15 10:38:41.449, 2024-01-15 10:38:41....",2024-01-29 20:18:51.948
2,"[outfit.3c4590746cbc4d0484357d425b40f653, outf...",outfit.0c22c8c8db0e4b469cb5433572cc6487,"[group.9ee65add32d241b9729a7237db884414, group...",group.3ab741dfab8f4ec1c2385b1df3638792,"[2022-06-01 23:16:59.078, 2022-06-01 23:16:59....",2022-06-01 23:16:59.078
3,[outfit.2392f0645935482bb171c7fd35384587],outfit.4eb647c9c034498da674fca7196fbccb,[group.95b244c3abe5d6b5552ef50448356d1a],group.60de1b488ecc65924d6714636e6f8ed5,[2022-07-31 20:31:01.840],2022-07-31 20:31:01.840
4,"[outfit.e377aedd50ec4073af1f18cb9eab432b, outf...",outfit.3f65ab1185ed4b06a4feab44d8fdc748,"[group.cbd88cf0356ef18209b64fb155b1879d, group...",group.0ae00b8afa85e2fd392ee557bf437c04,"[2022-07-31 20:04:08.529, 2022-07-31 20:04:08....",2022-08-17 20:23:29.483
...,...,...,...,...,...,...
2211,"[outfit.79ef6b130e3d4f46865ac114abfb7fcd, outf...",outfit.a8e66ebfb9004fa6ac6d7b8c46e381de,"[group.b6bb2312c4ab8357ba9c9bd2ed3c5c49, group...",group.3de77ccdf58eb30b09258b457729e0f2,"[2024-01-12 17:44:38.463, 2024-01-15 13:09:21....",2024-01-12 17:44:38.463
2212,"[outfit.da552355bfd04649a987a3881b99d09c, outf...",outfit.14727804dc6945f9baabcd76ad96a744,"[group.8068ba000114d2e1f247e469bab9e242, group...",group.0f3624e2fbc393dee85c7869bd896451,"[2022-08-21 07:58:42.868, 2022-08-21 07:58:42....",2022-08-22 14:27:00.167
2213,"[outfit.bb709dadbe604c4bb947559236ff8e07, outf...",outfit.83d18c06ab05475e84feb96a5b64eafe,"[group.8041d98fc65a186f3c94701cc1d365de, group...",group.077bfc8105470cc5b78b62f1ce27b31a,"[2023-08-31 19:16:11.997, 2023-08-31 19:16:11....",2023-09-12 11:16:42.831
2214,"[outfit.e80978de4a2a410c9ab14d3050fd9cf4, outf...",outfit.6b8aaed196c044848f417bc2da4314d0,"[group.51c6e2c0cae271a723afdfbeab59d2f1, group...",group.10e2052f38b5e8ee18aa6306bb7a6a9b,"[2021-12-06 18:32:21.698, 2021-12-06 18:51:54....",2022-02-08 21:05:20.159


In [24]:
import src.rs_methods
NUM_ITEMS = 10

# Most popular outfits prediction
# most_popular_train_outfit_ids, most_popular_train_groups = src.rs_methods.get_most_popular_outfits(user_splits_df, NUM_ITEMS)
# user_splits_df["id_prediction"] = [most_popular_train_outfit_ids] * len(user_splits_df)
# user_splits_df["group_prediction"] = [most_popular_train_groups] * len(user_splits_df)
# most_popular_train_outfit_ids, most_popular_train_groups = src.rs_methods.get_most_popular_outfits(user_splits_unique_df, NUM_ITEMS)
# user_splits_unique_df["id_prediction"] = [most_popular_train_outfit_ids] * len(user_splits_unique_df)
# user_splits_unique_df["group_prediction"] = [most_popular_train_groups] * len(user_splits_unique_df)


# Previous rental prediction
# user_splits_df["id_prediction"] = user_splits_df["train_outfit_ids"].apply(lambda x: x[-NUM_ITEMS:])
# user_splits_df["group_prediction"] = user_splits_df["train_group"].apply(lambda x: x[-NUM_ITEMS:])
# user_splits_unique_df["id_prediction"] = user_splits_unique_df["train_outfit_ids"].apply(lambda x: x if len(x) <= NUM_ITEMS else x[-NUM_ITEMS:])
# user_splits_unique_df["group_prediction"] = user_splits_unique_df["train_group"].apply(lambda x: x if len(x) <= NUM_ITEMS else x[-NUM_ITEMS:])

# Previous rental + most popular outfits prediction
def pad_with_most_popular(x, pop_outfits):
    if len(x) < NUM_ITEMS:
        return np.append(x, pop_outfits[:NUM_ITEMS - len(x)])
    else:
        return x[-NUM_ITEMS:]

most_popular_train_outfit_ids, most_popular_train_groups = src.rs_methods.get_most_popular_outfits(user_splits_df, NUM_ITEMS)
user_splits_df["id_prediction"] = user_splits_df.apply(lambda x: pad_with_most_popular(x["train_outfit_ids"], most_popular_train_outfit_ids), axis=1)
user_splits_df["group_prediction"] = user_splits_df.apply(lambda x: pad_with_most_popular(x["train_group"], most_popular_train_groups), axis=1)
user_splits_unique_df["id_prediction"] = user_splits_unique_df.apply(lambda x: pad_with_most_popular(x["train_outfit_ids"], most_popular_train_outfit_ids), axis=1)
user_splits_unique_df["group_prediction"] = user_splits_unique_df.apply(lambda x: pad_with_most_popular(x["train_group"], most_popular_train_groups), axis=1)

# Random prediction
# def get_random_outfits(x):
#     return np.random.choice(all_outfit_ids, NUM_ITEMS, replace=False)
# all_outfit_ids = outfits_df["id"].values
# all_groups = outfits_df["group"].values
# user_splits_df["id_prediction"] = user_splits_df.apply(lambda x: get_random_outfits(x), axis=1)
# user_splits_df["group_prediction"] = user_splits_df.apply(lambda x: np.random.choice(all_groups, NUM_ITEMS, replace=False), axis=1)
# user_splits_unique_df["id_prediction"] = user_splits_unique_df.apply(lambda x: get_random_outfits(x), axis=1)
# user_splits_unique_df["group_prediction"] = user_splits_unique_df.apply(lambda x: np.random.choice(all_groups, NUM_ITEMS, replace=False), axis=1)

In [63]:
# Test if test outfit is in train outfits for recommendation of repeated outfits
# def check_if_test_in_train(train_outfit_ids, test_outfit_id):
#     return test_outfit_id in train_outfit_ids

# user_splits_df["test_in_train"] = user_splits_df.apply(lambda x: check_if_test_in_train(x["train_outfit_ids"], x["test_outfit_id"]), axis=1)
# user_splits_unique_df["test_in_train"] = user_splits_unique_df.apply(lambda x: check_if_test_in_train(x["train_outfit_ids"], x["test_outfit_id"]), axis=1)
# display(user_splits_df["test_in_train"].value_counts())
# user_splits_df[["train_outfit_ids", "test_outfit_id", "id_prediction", "test_in_train"]]
# display(user_splits_unique_df["test_in_train"].value_counts())
# user_splits_unique_df[["train_outfit_ids", "test_outfit_id", "id_prediction", "test_in_train"]]


# user_splits_unique_df["train_length"] = user_splits_unique_df["train_outfit_ids"].apply(len)
# user_splits_unique_df["prediction_length"] = user_splits_unique_df["id_prediction"].apply(len)

In [25]:
from IPython.display import display

def evaluate_hit_rate_at_n(test_id, predicted_ids, n=10):
    predicted_ids = predicted_ids[:n]
    if test_id in predicted_ids:
        #print(f"Hit at {n} for {test_id} in {predicted_ids}")
        return 1
    return 0

def evaluate_df_hit_rate_at_n(df, n=10):
    HIT_RATE_COLUMNS = ["id_hit_rate_at_100", "id_hit_rate_at_10", "group_hit_rate_at_100", "group_hit_rate_at_10"]
    df["id_hit_rate_at_100"] = df.apply(lambda x: evaluate_hit_rate_at_n(x["test_outfit_id"], x["id_prediction"], n=100), axis=1)
    df["id_hit_rate_at_10"] = df.apply(lambda x: evaluate_hit_rate_at_n(x["test_outfit_id"], x["id_prediction"], n=10), axis=1)
    df["group_hit_rate_at_100"] = df.apply(lambda x: evaluate_hit_rate_at_n(x["test_group"], x["group_prediction"], n=100), axis=1)
    df["group_hit_rate_at_10"] = df.apply(lambda x: evaluate_hit_rate_at_n(x["test_group"], x["group_prediction"], n=10), axis=1)
    display(df[HIT_RATE_COLUMNS].mean())
    result_dict = {column: df[column].mean() for column in HIT_RATE_COLUMNS}
    return df, result_dict


user_splits_df, all_dict = evaluate_df_hit_rate_at_n(user_splits_df, n=10)
user_splits_unique_df, ind_dict = evaluate_df_hit_rate_at_n(user_splits_unique_df, n=10)

id_hit_rate_at_100       0.033845
id_hit_rate_at_10        0.033845
group_hit_rate_at_100    0.046931
group_hit_rate_at_10     0.046931
dtype: float64

id_hit_rate_at_100       0.000903
id_hit_rate_at_10        0.000903
group_hit_rate_at_100    0.005869
group_hit_rate_at_10     0.005869
dtype: float64

In [27]:
import pyperclip

def format_dicts_into_latex(all_dict, ind_dict, precision=4, run_name="Random"):
    first_row = f"{run_name} Ind: & {all_dict['id_hit_rate_at_10']:.{precision}f} & {all_dict['id_hit_rate_at_100']:.{precision}f} & {ind_dict['id_hit_rate_at_10']:.{precision}f} & {ind_dict['id_hit_rate_at_100']:.{precision}f} \\\\"
    second_row = f"{run_name} Groups: & {all_dict['group_hit_rate_at_10']:.{precision}f} & {all_dict['group_hit_rate_at_100']:.{precision}f} & {ind_dict['group_hit_rate_at_10']:.{precision}f} & {ind_dict['group_hit_rate_at_100']:.{precision}f} \\\\\\hline"
    full_string = first_row + "\n" + second_row + "\n"
    print(full_string)
    pyperclip.copy(full_string)

format_dicts_into_latex(all_dict, ind_dict, precision=4, run_name="Rep + Pop")

Rep + Pop Ind: & 0.0338 & 0.0338 & 0.0009 & 0.0009 \\
Rep + Pop Groupe: & 0.0469 & 0.0469 & 0.0059 & 0.0059 \\\hline



In [109]:
import numpy as np

def get_outfit_category(tag_categories, tags, category):
    tag_categories, tags = np.array(tag_categories), np.array(tags)
    category_indexes = np.where(tag_categories == category)[0]
    if len(category_indexes) == 0:
        return ""
    cat_tags = tags[category_indexes]
    output = str(cat_tags[0])
    return output

outfits_df["size"] = outfits_df.apply(lambda x: get_outfit_category(x["tag_categories"], x["outfit_tags"], "Size"), axis=1)


In [78]:
outfits_df

,id,name,description,group,owner,timeCreated,retailPrice,outfit_tags,tag_categories,size
0,outfit.fffdaa715c3646f8b1c0f04d549ff07e,Out of stock - Asymmetric Frilled Dress,"This fun, short dress features and asymmetric ...",group.50a586c78eb7626e294ba3bd07d12c79,464,2017-12-30 11:28:01.000,4000.0,"[Metallic, Synthetic, Cotton, Sandro, Dresses,...","[Details, Material, Material, Brand, Category,...",S
1,outfit.fffa1b9a3db6415d806f3c48f8ab58d9,Yellow Shell Mellomholmene Blouse,This beautiful blouse features an adjustable n...,group.61ad2fcabb3e9197e3836376e6b67f2c,112,2021-06-07 12:07:22.921,1300.0,"[Yellow, Cotton, Blouses, Everyday, M, Summer,...","[Color, Material, Category, Occasion, Size, Se...",M
2,outfit.fff175b13ceb453f9928625491412ede,Kaula Dress Black,Kaula from Rodebjer is a fitted dress made in ...,group.37c2b59d63d3a9c2d58e07f532f71f7f,635,2023-06-05 09:17:59.004,3100.0,"[Synthetic, Multi Season, Rodebjer, Everyday, ...","[Material, Seasons, Brand, Occasion, Size, Cat...",M
3,outfit.ffef9d7c292a48b69076d2df2e32352f,For sale - Jarvis Blouse,This wrap blouse has mid length sleeves and a ...,group.dfcaa57546b0b7a5e9eb204449b6cc1c,745,2021-05-18 14:02:28.690,1500.0,"[Cotton, Multi Season, Floral, Wrap, XS, Style...","[Material, Seasons, Details, Fit, Size, Brand,...",XS
4,outfit.ffeef842238f4dbdabc6c730a75aa2bd,Black Amber Pants,"Feel slack and nice dressed with this pant, ma...",group.ee297c977905eb21a123a4aea5fbb6d2,504,2021-07-16 14:02:30.643,1200.0,"[Winter, Cotton, L, Knitwear, Everyday, Fall, ...","[Seasons, Material, Size, Category, Occasion, ...",L
...,...,...,...,...,...,...,...,...,...,...
15824,outfit.001bf665330140cf854dcfb1cbff6b5f,Out of stock - Harley Vintage White Midi Dress,This gorgeous dress is cut in the most flatter...,group.d91a2a6728833c8082dadf27b95488a9,140,2019-06-25 10:13:55.000,3800.0,"[Viscose, L, Midi, Dresses, White, Formal, Pia...","[Material, Size, Length, Category, Color, Occa...",L
15825,outfit.0018701ce6b049ebadc314d16623caa8,Vintage Burberry Trench Coat,You really can't go wrong with this Classic Tr...,group.6be510229d0f9faf5d19d52e7e2b2a95,58,2023-02-07 07:54:06.214,22000.0,"[Winter, Cotton, Midi, Everyday, Fall, Burberr...","[Seasons, Material, Length, Occasion, Seasons,...",S
15826,outfit.0014a5c89b244077a3d7cffd4549718e,Mira Skirt Brown,The Mira Skirt in Brown from Stine Goya is an ...,group.668be5db7976aa2cb9213dd4c7f9b7fe,4,2023-10-09 09:12:14.631,1500.0,"[Viscose, Midi, Skirts, Summer, Stine Goya, Ev...","[Material, Length, Category, Seasons, Brand, B...",S
15827,outfit.0013691ff35b440e9dcfe1748ec184c7,Oldina Parka Cotta,The Oldina Parka from Kari Traa is a women's p...,group.c82046bcba672c8ec9b21be4f844b402,552,2023-02-23 12:20:27.042,3500.0,"[Winter, Synthetic, Midi, Everyday, XS, Coats,...","[Seasons, Material, Length, Occasion, Size, Ca...",XS
